In [ ]:
# 找出在cake人力銀行中，搜尋「資料工程師」的職缺資料
# 舊的，於10/30爬蟲...單純抓搜尋"資料工程師"，未特別設定職類。
# 有多餘的程式碼，再次爬蟲時有進一步減少程序。

import urllib.request as req
import bs4 as bs

import pandas as pd

import time

# 嘗試處理，抓到後面沒資料的話，出現http 404，設定第二組try...except
import requests
from urllib.error import HTTPError

# 準備一個空白的list
table = [] # Clear the table list before scraping

# 抓100頁資料，如遇問題可繼續，並會顯示錯誤為何（比如IndexError或是too many requests）
for i in range(100):
    page = i + 1
    # Corrected URL to include page number，需要原本的網址後面加上?page=
    url = "https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=" + str(page)
    print("頁數:", url)
    # 添加了對request清單長度的檢查，以防止任何職位清單未包含所有預期元素時出現潛在的 IndexError 錯誤。
    # 添加了 try...except 程式碼區塊，用於捕獲抓取過程中的任何其他潛在錯誤，並允許程式碼繼續處理後續頁面。

    time.sleep(3)
    # 使用 try，測試內容是否正確
    try:
        resp = req.urlopen(url)
        content = resp.read()
        html = bs.BeautifulSoup(content)

        # find和find_all是區塊的專屬功能，抓取一個div（區塊）有哪些符合搜尋條件的，而其產出的狀態為 list。
        # js為jobs（多個），j為job（單個）
        js = html.find_all("div", {"class":"JobSearchItem_container__oKoBL"})
        # 以上find_all萃取出 所有職缺區塊
        for j in js:
            # 欲抓取的資料
            t = j.find("a", {"class":"JobSearchItem_jobTitle__bu6yO"})# t為職缺名稱
            c = j.find("a", {"class":"JobSearchItem_companyName__bY7JI"})# c為公司名稱
            request = j.find_all("div", {"class":"InlineMessage_label__LJGjW"})# 表需要的員工條件，有多個資料共用同一個區塊

            # 在訪問所需元素之前，檢查它們是否存在
            # 此部分為解決依課程方式抓m會出現「IndexError: list index out of range」
            # 以下if...至else為colab的gemini修改產生，何時要設text何時不用設，該邏輯尚須研究
            # 邏輯初步猜測：薪水和工作經驗是敘述性，需求人數、工作地點、管理責任是標籤？
            if len(request) > 4:
                jo = request[0]# jo為需求人數，job opening
                l = request[1]# l為工作地點
                s_text = request[2].get_text().strip()# s為薪資範圍
                ex_text = request[3].get_text().strip()# ex為工作年資
                m = request[4]# m為管理責任
            else:
                # 處理某些元素缺失的情況
                jo = None
                l = None
                s_text = None
                ex_text = None
                m = None

            # m = resquest[4]# m為管理責任，此階段出現「IndexError: list index out of range」
            jd = j.find("div",{"class":"JobSearchItem_description__si5zg"})# Jd為工作說明



            # 欲產生的資料
            # 以下if和else為chatgpt產生，該邏輯尚須研究
            t_text = t.get_text().strip() if t else None
            c_text = c.get_text().strip() if c else None
            jo_text = jo.get_text().strip() if jo else None
            l_text = l.get_text().strip() if l else None
            # s = s.get_text().strip() 原本設定要做，但系統直接將text設定在上面，可忽略
            # ex = ex.get_text().strip() 原本設定要做，但系統直接將text設定在上面，可忽略
            m = m.get_text().strip() if m else None
            jd_text = jd.get_text().strip() if jd else None
            t_href = "www.cake.me/" + t["href"] if t and "href" in t.attrs else None

            # 一筆資料就是一個字典
            row = {
                "職務名稱:":t_text,
                "公司名稱:":c_text,
                "需求人數:":jo_text,
                "工作地點:":l_text,
                "薪資範圍:":s_text,
                "工作年資:":ex_text,
                "管理責任:":m,
                "工作說明:":jd_text,
                "網站連結:":t_href,
            }
            table.append(row)

            # 呈現的資料（前測檢視用）
            # print("職務名稱:",t_text)
            # print("公司名稱:",c_text)
            # print("需求人數:",jo_text)
            # print("工作地點:",l_text)
            # print("薪資範圍:",s)
            # print("工作年資:",ex)
            # print("管理責任:",m)
            # print("工作說明:",jd_text)
            # print("網站連結:",t_href)
            # 分隔線
            # print("-" * 30)
    # 如果 try 的內容發生錯誤，就執行 except 裡的內容
    except Exception as e:
        print(f"第 {page} 頁發生錯誤：{e}")
        # 如果出現錯誤，還是繼續下一頁
        # 使用 f-string 將變數 page 插入字串並印出，讓使用者知道第幾頁出現什麼錯誤
        # 該指令等同「print("第" + str(page) + "頁發生錯誤：" + str(e))」


# 進一步顯示是否有成功抓取或失敗，抓到後面頁數沒資料的話，便會停止爬蟲
    # 以下為chatgpt提供之作法，嘗試成功
    # 使用 try，測試內容是否正確
    try:
        response = requests.get(url) # 套用requests第三方套件，帶入欲查詢的網址
        response.raise_for_status()  # 若不是HTTP 200，會觸發HTTPError（觸發Exception）。
        data = response.text # 取得回應內容並以字串型態存到data。 requests會試圖自動偵測編碼並進行解碼。

        # 若頁面有明顯「無資料(頁面文字包含no jobs found或字元數小於 500 → 視為空頁）」則停止爬取
        if "No jobs found" in data or len(data) < 500:
            print(f"第 {page} 頁沒有資料，停止爬取。")
            # 使用 f-string 將變數 page 插入字串並印出，讓使用者知道第幾頁無資料
            # 該指令等同「print("第" + str(page) + "頁沒有資料，停止爬取。")」
            break
            # 沒有資料便break

        print(f"成功抓取第 {page} 頁")
        # 使用 f-string 將變數 page 插入字串，有資料且成功抓取就會印出
        # 該指令等同「print("成功抓取第" + str(page) + "頁")」

    # 如果 try 的內容發生錯誤，就執行 except 裡的內容
    # 捕捉 response.raise_for_status() 所拋出的 HTTPError
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404: # 伺服器給你的回應狀態（整數，例如 200、404）
        # 若狀態碼為404（頁面不存在），表示可能已到最後一頁、或分頁號碼超過範圍
            print(f"第 {page} 頁不存在（404），停止爬取。")
            # 使用 f-string 將變數 page 插入字串，讓使用者知道第幾頁不存在（404）
            # 該指令等同「print("第" + str(page) + "頁不存在（404），停止爬取。")」
            break
        else:
            continue

# 轉換存檔
df = pd.DataFrame(table)
df.to_csv("cakejob.csv", encoding="utf-8")
df

頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=1
成功抓取第 1 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=2
成功抓取第 2 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=3
成功抓取第 3 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=4
成功抓取第 4 頁
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=5
第 5 頁發生錯誤：HTTP Error 404: Not Found
第 5 頁不存在（404），停止爬取。


,職務名稱:,公司名稱:,需求人數:,工作地點:,薪資範圍:,工作年資:,管理責任:,工作說明:,網站連結:
0,IT Data Engineer,TSMC 台積電,1・Full-time・Entry level,"Hsinchu, North District, Hsinchu City, Taiwan・...",40K+ TWD / month,No requirement for relevant working experience,No management responsibility,Established in 1987 and headquartered in Taiwa...,www.cake.me//companies/tsmc/jobs/data-engineer...
1,數據工程師實習生 Data Engineer Intern,COMMEET＿擁樂數據服務股份有限公司,1・Internship・Entry level,"Songshan District, Taipei City, Taiwan 105・Tai...",200 ~ 250 TWD / hour,No requirement for relevant working experience,No management responsibility,COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...,www.cake.me//companies/commeet/jobs/data-assis...
2,資料工程師,上洋股份有限公司,1・Full-time・Assistant,"Wugu District, New Taipei, Taiwan",50K+ TWD / month,2 years of experience required,No management responsibility,數據驅動決策與營運最佳化是上洋邁向智慧營運的重要一步。我們正在尋找一位對資料工程具備熱情與專...,www.cake.me//companies/upyoung/jobs/data-engin...
3,[Bosch R&D - Internship] Data Engineer Intern,Bosch Vietnam,None,None,None,None,None,Company Description\nBosch first started in Vi...,www.cake.me//companies/bosch-vietnam-bd2881/jo...
4,【國泰集團】數據工程師 Data Engineer,國泰健康管理顧問股份有限公司,1・Full-time・Entry level,"Daan District, Taipei City, Taiwan",800K ~ 1M TWD / year,3 years of experience required,No management responsibility,【About Us】\n歡迎加入「國泰健康管理」！我們是一個充滿活力、追求創新、以數據和科技...,www.cake.me//companies/cathay-hcm/jobs/cathay-...
5,Python 工程師 – 資料科學/AI 應用方向 【Munger AI Team】,Stark Tech 鷹翔有限公司,3・Full-time・Mid-Senior level,"Songshan District, Taipei City, Taiwan",850K ~ 1.2M TWD / year,3 years of experience required,No management responsibility,【關於我們 - Data Munger AI】\nData Munger 致力成為企業與實用...,www.cake.me//companies/StarkTech/jobs/python-e...
6,數據工程師 Data Science Engineer,COMMEET＿擁樂數據服務股份有限公司,1・Full-time・Assistant,"Songshan District, Taipei City, Taiwan",Negotiable,No requirement for relevant working experience,No management responsibility,COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...,www.cake.me//companies/commeet/jobs/data-scien...
7,資料工程師,群健科技股份有限公司Cofit Healthcare Inc.,2・Full-time・Mid-Senior level,"Taipei City, Taiwan",70K ~ 100K TWD / month,4 years of experience required,No management responsibility,★ 關於 Cofit\nHi，我們是 Cofit，正在打造一個橫跨台灣、美加、新加坡、香港、...,www.cake.me//companies/cofit-healthcare-inc/jo...
8,Data Engineer / 資料工程師,SWAG,1・Full-time・Mid-Senior level,"Taipei City, Taiwan",Negotiable,3 years of experience required,No management responsibility,"DescriptionAt SWAG Live, data is the backbone ...",www.cake.me//companies/swag/jobs/data-team-eng...
9,【風控資料工程師】Data Engineer(Risk Analytics),智鈦星平台服務有限公司,1・Full-time・Entry level,"Taipei City, Taiwan",60K ~ 120K TWD / month,2 years of experience required,No management responsibility,身為風控數據工程師，你將負責設計、建置與維運公司的數據基礎設施（Data Platform）...,www.cake.me//companies/gtstw/jobs/data-profess...


In [29]:
# 找出在cake人力銀行中，搜尋「資料工程師」的職缺資料

# 新的，於11/18爬蟲...除了抓搜尋"資料工程師"，也有設定 軟體 > 數據工程師 的職類來嘗試。
# 相較於舊版，針對「人力需求」的細項進一步萃取。（55行起)
# 後續優化評估：是否要針對更多職類來抓？目前僅能在 professions 找到職類分類，要如何抓到df中（分析不同職類的差異）？

from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import requests as req
from urllib.error import HTTPError

# 設定 headers 模擬瀏覽器，因應反爬蟲
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# 準備一個空白的list
table = [] # Clear the table list before scraping

# 抓100頁資料，如遇問題可繼續，並會顯示錯誤為何（比如IndexError或是too many requests）
for i in range(100):
    page = i + 1
    url = "https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=" + str(page)
    print("頁數:", url)
    time.sleep(3)
    # 使用 try-except，當 try 內的程式發生被捕捉的例外（Exception）時，會跳到相對應的 except 分支處理，不會直接讓程式崩掉
    try:
        resp = req.get(url)
        resp.raise_for_status()  # 若不是HTTP 200的狀態（代表伺服器成功回傳資料），會觸發HTTPError（觸發Exception）。
        html = bs(resp.text, "html.parser") # html.parser 可把 HTML 字串「解析」成可操作的節點（tags）、屬性（attributes）、文字（text）等
        data = resp.text # 取得回應內容並以字串型態存到data。 requests會試圖自動偵測編碼並進行解碼。

        # 進一步顯示是否有成功抓取或失敗，抓到後面頁數沒資料的話，便會停止爬蟲
        # 若頁面有明顯「無資料(頁面文字包含no jobs found或字元數小於 500 → 視為空頁）」則停止爬取
        if "No jobs found" in data or len(data) < 500:
            print(f"第 {page} 頁沒有資料，停止爬取。")
            # 使用 f-string 將變數 page 插入字串並印出，讓使用者知道第幾頁無資料
            # 該指令等同「print("第" + str(page) + "頁沒有資料，停止爬取。")」
            break
            # 沒有資料便break

        # find和find_all是區塊的專屬功能，抓取一個div（區塊）有哪些符合搜尋條件的，而其產出的狀態為 list。
        # js為jobs（多個），j為job（單個）
        js = html.find_all("div", {"class":"JobSearchItem_container__oKoBL"})
        # 以上find_all萃取出 所有職缺資訊區塊
        for j in js:
            # 欲抓取的資料
            t = j.find("a", {"class":"JobSearchItem_jobTitle__bu6yO"})# title為職缺名稱
            c = j.find("a", {"class":"JobSearchItem_companyName__bY7JI"})# company為公司名稱

            lebals = j.find_all("div", {"class":"InlineMessage_label__LJGjW"})# 表需要的員工條件，有多個資料共用同一個區塊

            # 直接結合抓取資料和轉換成文字。
            # 透過以下方式逐項取出資料（含安全檢查），如果list裡至少有 N 個元素才會取出第 N 個，若無 → 設為 None（避免 IndexError）
            # 用更白話的方式比喻：這段程式碼＝「我去便利商店買便當（抓資料）。如果店裡有雞腿飯，我就買雞腿飯；如果沒有，我就買不到（None），但我不會生氣（error）離開。」
            hr = lebals[0].get_text().strip() if len(lebals) >= 1 else None  # human request人力需求（含職務類別/等級）
            l_text = lebals[1].get_text().strip() if len(lebals) >= 2 else None #location
            s_text = lebals[2].get_text().strip() if len(lebals) >= 3 else None #salary
            ex_text = lebals[3].get_text().strip() if len(lebals) >= 4 else None #experience
            m_text = lebals[4].get_text().strip() if len(lebals) >= 5 else None #management

            jd = j.find("div",{"class":"JobSearchItem_description__si5zg"})# Jd為工作說明

            # 新增：拆解 hr → 招募人數 / 職務類別 / 職務等級
            # 初步萃取 hr 字串時，會有上面三個子資料，並以・做為區隔（見舊的版本所產生的"需求人數"欄位）
            hn_text, jt_text, jl_text = None, None, None # 一次設定三個變數預設為 None，避免後面沒資料時程式噴錯。
            if hr and "・" in hr: # hr 字串裡如有 "・" 這個符號，將繼續往下解析
                parts = hr.split("・") # parts為文字字串，並以・拆分
                if len(parts) >= 1: hn_text = parts[0] # hire number 招募人數
                if len(parts) >= 2: jt_text = parts[1] # job_type 職務類別
                if len(parts) >= 3: jl_text = parts[2] # job_level 職務等級

            # 將前面未設定的資料轉換文字，如果沒有則為none
            t_text = t.get_text().strip() if t else None
            c_text = c.get_text().strip() if c else None
            jd_text = jd.get_text().strip() if jd else None
            # 檢查這個 tag 是否有 href ，如有責組成一個連結，否則為none
            t_href = "www.cake.me/" + t["href"] if t and "href" in t.attrs else None

            # 一筆資料就是一個字典
            row = {
                "職務名稱:":t_text,
                "公司名稱:":c_text,
                "招募人數:":hn_text,
                "職務類別:":jt_text,
                "職務等級:":jl_text,
                "工作地點:":l_text,
                "薪資範圍:":s_text,
                "工作年資:":ex_text,
                "管理責任:":m_text,
                "工作說明:":jd_text,
                "網站連結:":t_href,
            }
            table.append(row)

        print(f"成功抓取第 {page} 頁，共 {len(js)} 筆")
        # 使用 f-string 將變數 page 插入字串，有資料且成功抓取就會印出
        # 該指令等同「print("成功抓取第" + str(page) + "頁，共" + str(len(js)) + "筆")」

    # 如果 try 的內容發生錯誤，就執行 except 裡的內容
    except req.exceptions.HTTPError as e:
        if e.response.status_code == 404: # 伺服器給你的回應狀態為404（頁面不存在），表示可能已到最後一頁、或分頁號碼超過範圍
            print(f"第 {page} 頁不存在（404），停止爬取。")
            # 使用 f-string 將變數 page 插入字串，讓使用者知道第幾頁不存在（404）
            # 該指令等同「print("第" + str(page) + "頁不存在（404），停止爬取。")」
            break
        else:
            continue

# 轉換存檔
df = pd.DataFrame(table)
df.to_csv("cakejob_new.csv", encoding="utf-8")
df

頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=1
成功抓取第 1 頁，共 10 筆
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=2
成功抓取第 2 頁，共 10 筆
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=3
成功抓取第 3 頁，共 10 筆
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=4
成功抓取第 4 頁，共 7 筆
頁數: https://www.cake.me/jobs/%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB?professions%5B0%5D=it_data-engineer&page=5
第 5 頁不存在（404），停止爬取。


,職務名稱:,公司名稱:,招募人數:,職務類別:,職務等級:,工作地點:,薪資範圍:,工作年資:,管理責任:,工作說明:,網站連結:
0,IT Data Engineer,TSMC 台積電,1,Full-time,Entry level,"Hsinchu, North District, Hsinchu City, Taiwan・...",40K+ TWD / month,No requirement for relevant working experience,No management responsibility,Established in 1987 and headquartered in Taiwa...,www.cake.me//companies/tsmc/jobs/data-engineer...
1,數據工程師實習生 Data Engineer Intern,COMMEET＿擁樂數據服務股份有限公司,1,Internship,Entry level,"Songshan District, Taipei City, Taiwan 105・Tai...",200 ~ 250 TWD / hour,No requirement for relevant working experience,No management responsibility,COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...,www.cake.me//companies/commeet/jobs/data-assis...
2,資料工程師,上洋股份有限公司,1,Full-time,Assistant,"Wugu District, New Taipei, Taiwan",50K+ TWD / month,2 years of experience required,No management responsibility,數據驅動決策與營運最佳化是上洋邁向智慧營運的重要一步。我們正在尋找一位對資料工程具備熱情與專...,www.cake.me//companies/upyoung/jobs/data-engin...
3,[Bosch R&D - Internship] Data Engineer Intern,Bosch Vietnam,5,Full-time,Internship,"Ho Chi Minh City, Vietnam",No requirement for relevant working experience,No management responsibility,None,Company Description\nBosch first started in Vi...,www.cake.me//companies/bosch-vietnam-bd2881/jo...
4,【國泰集團】數據工程師 Data Engineer,國泰健康管理顧問股份有限公司,1,Full-time,Entry level,"Daan District, Taipei City, Taiwan",800K ~ 1M TWD / year,3 years of experience required,No management responsibility,【About Us】\n歡迎加入「國泰健康管理」！我們是一個充滿活力、追求創新、以數據和科技...,www.cake.me//companies/cathay-hcm/jobs/cathay-...
5,Python 工程師 – 資料科學/AI 應用方向 【Munger AI Team】,Stark Tech 鷹翔有限公司,3,Full-time,Mid-Senior level,"Songshan District, Taipei City, Taiwan",850K ~ 1.2M TWD / year,3 years of experience required,No management responsibility,【關於我們 - Data Munger AI】\nData Munger 致力成為企業與實用...,www.cake.me//companies/StarkTech/jobs/python-e...
6,數據工程師 Data Science Engineer,COMMEET＿擁樂數據服務股份有限公司,1,Full-time,Assistant,"Songshan District, Taipei City, Taiwan",Negotiable,No requirement for relevant working experience,No management responsibility,COMMEET (https://commeet.co) 擴大徵才，歡迎有熱情之新夥伴加入我...,www.cake.me//companies/commeet/jobs/data-scien...
7,資料工程師,群健科技股份有限公司Cofit Healthcare Inc.,2,Full-time,Mid-Senior level,"Taipei City, Taiwan",70K ~ 100K TWD / month,4 years of experience required,No management responsibility,★ 關於 Cofit\nHi，我們是 Cofit，正在打造一個橫跨台灣、美加、新加坡、香港、...,www.cake.me//companies/cofit-healthcare-inc/jo...
8,Data Engineer / 資料工程師,SWAG,1,Full-time,Mid-Senior level,"Taipei City, Taiwan",Negotiable,3 years of experience required,No management responsibility,"DescriptionAt SWAG Live, data is the backbone ...",www.cake.me//companies/swag/jobs/data-team-eng...
9,【風控資料工程師】Data Engineer(Risk Analytics),智鈦星平台服務有限公司,1,Full-time,Entry level,"Taipei City, Taiwan",60K ~ 120K TWD / month,2 years of experience required,No management responsibility,身為風控數據工程師，你將負責設計、建置與維運公司的數據基礎設施（Data Platform）...,www.cake.me//companies/gtstw/jobs/data-profess...


🎯 初學者建議-bs的解析器

html.parser 是 BeautifulSoup 解析 HTML 時所使用的一種「解析器（parser）」。

你可以把 BeautifulSoup 想像成一個「整理 HTML 的工具」，但它本身不能直接讀懂 HTML，它需要「解析器」來協助拆解 HTML 樹狀結構。

它決定 BeautifulSoup 要用什麼方式把 HTML 字串解析成可操作的節點（tags）、屬性（attributes）、文字（text）等。就像你要讀 Word 檔，需要 Word 或其他能開啟 .docx 的軟體。

如果你是初階爬蟲使用者：

👉 直接用 html.parser 完全沒問題。

如果遇到頁面解析不出資料，再考慮切換：

```
soup = BeautifulSoup(html_text, "lxml")      # 最快
soup = BeautifulSoup(html_text, "html5lib")  # 解析最準

```



In [ ]:
pip install selenium beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 28.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful